In [19]:
import sys
!{sys.executable} -m pip install GitPython langchain-openai langchain-core

In [13]:
from datetime import date
from typing import List
from langchain_core.tools import tool
from git import Git

repo_directory = './defects4j'

@tool
def list_commits_between(start_date: date, end_date: date) -> str:
    """ Returns the messages of the commits from an interval between dates.
    Args:
        start_date: The start of the interval.
        end_date: The end of the date.
    """
    repo = Git(repo_directory)
    logs = repo.log('--since={}'.format(start_date), '--until={}'.format(end_date), '--oneline')
    return logs

In [4]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [6]:
llm_with_tools = llm.bind_tools([list_commits_between])

In [7]:
query = 'Summarize what has been done in the project last year.'

In [14]:
msg_with_tool_calls = llm_with_tools.invoke(query)
msg_with_tool_calls

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jfAmlpAMNPyUjLdjKHERxiF0', 'function': {'arguments': '{"start_date":"2022-10-01","end_date":"2023-10-01"}', 'name': 'list_commits_between'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 103, 'total_tokens': 136, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a340668a-157b-4e96-805d-7d182668c33b-0', tool_calls=[{'name': 'list_commits_between', 'args': {'start_date': '2022-10-01', 'end_date': '2023-10-01'}, 'id': 'call_jfAmlpAMNPyUjLdjKHERxiF0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 103, 'output_tokens': 33, 'total_tokens': 136, 'input_token_

In [15]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(query), msg_with_tool_calls]

for tool_call in msg_with_tool_calls.tool_calls:
    selected_tool = locals()[tool_call['name']]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='Summarize what has been done in the project last year.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jfAmlpAMNPyUjLdjKHERxiF0', 'function': {'arguments': '{"start_date":"2022-10-01","end_date":"2023-10-01"}', 'name': 'list_commits_between'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 103, 'total_tokens': 136, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a340668a-157b-4e96-805d-7d182668c33b-0', tool_calls=[{'name': 'list_commits_between', 'args': {'start_date': '2022-10-01', 'end_date': '2023-10-01'}, 'id': 'call_jfAmlpAMNPyUjLdj

In [17]:
result = llm_with_tools.invoke(messages)
result.pretty_print()

================================== Ai Message ==================================

Over the past year, the project has seen several important updates and fixes, including:

1. **Error Handling Improvements**: 
   - Fixed the number of uncompilable tests by adjusting the error reporting of compilation errors to ensure each test is only assessed once.
   - Added a debugging command-line argument to enhance diagnostics.

2. **Merge and Issue Fixes**:
   - Merged pull requests and fixed incorrect links to Mockito issues.
   - Clarified bugs within the code, addressing specific issues with the Math 99 target names.

3. **Dependency Management**:
   - Excluded certain ant dependencies from the JxPath classpath to prevent common execution errors during testing.

4. **Test Performance Enhancements**:
   - Implemented fixes for testing failures related to logical operators, specifically replacing logical or with logical defined-or.
   - Adjusted test oracles to ensure correct handling of test ID

## Exercise

1. Modify the code to ask about a specific contributor.